In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [62]:
import os
os.chdir("/content/drive/MyDrive/1.letnik MAG/NLP")

In [63]:
import os
os.chdir("/content/drive/MyDrive/1.letnik MAG/NLP/NLP-projekt/Data")

In [64]:
import pandas as pd
df = pd.read_csv('./Annotated/EN/TERM_CAT_GEN_SENT_REL.csv')
df

,DEFINIENDUM,CATEGORY,CATEGORY_TEXT,SENTENCE,*,AFFECTS,COMPOSITION_MEDIUM,CONTAINS,DEFINED_AS,DEFINITOR,...,frame_FORM,frame_FUNCTION,frame_LOCATION,frame_MEASURES,frame_MEDIUM,frame_POSITION,frame_RESULT,frame_SIZE,frame_STUDIES,frame_TIME
0,Karst,C. Geome|D.1 Abiotic|C. Geome|C. Geome|B.2 Loss,volcanic and permafrost areas|carbonate rocks|...,Karst is a landscape created by the dissolutio...,NaN,NaN,NaN,NaN,NaN,is a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Karst,C. Geome|D.1 Abiotic|C. Geome|B. Process,continental land masses|sea level|Karst|processes,Karst is a product of processes that operate o...,NaN,NaN,NaN,NaN,NaN,is a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Karst,B.2 Loss|D.3 Property|D.1 Abiotic|D.1 Abiotic|...,solution channel ( secondary ) porosity|rock s...,Karst is terrain with distinctive hydrology an...,NaN,NaN,NaN,with distinctive hydrology and landforms,NaN,is,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Karst,C. Geome|D.1 Abiotic|D.1 Abiotic|D.1 Abiotic|C...,underground water systems|soluble rocks|limest...,Karst is the term used to describe a special s...,NaN,NaN,developed on especially soluble rocks such as ...,containing caves and extensive underground wat...,NaN,is the term used to describe a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Karst conduit networks,C. Geome|C. Geome|D.1 Abiotic,Karst conduit networks|karst massif|conduits,“ Karst conduit networks ” are defined as the ...,NaN,NaN,NaN,NaN,the total length of the existing conduits with...,are defined as,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wider than,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,syngenetic karst,C. Geome|D.1 Abiotic|B. Process|B.4 Transforma...,syngenetic karst|aeolian calcarenite|evolution...,syngenetic karst - karst developed in aeolian ...,NaN,NaN,developed in aeolian calcarenite,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,when
839,tafoni,A.1 Surface landform,tafoni,tafoni - roughly hemispherical hollows weather...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,at the|in,NaN,NaN,NaN,NaN,NaN,NaN,NaN
840,through cave,A.2 Underground landform|D.1 Abiotic|D.1 Abiot...,through cave|stream course|stream|passage|cave,through cave - a cave which may be followed fr...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,which may be,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
841,tower karst,A.1 Surface landform|A.1 Surface landform|C. G...,alluvial plains|flat-floored depressions|tower...,tower karst - conekarst in which the residual ...,NaN,NaN,NaN,there my be alluvial plains between the towers...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
!pip install stanza
import stanza

In [66]:
nlp = stanza.Pipeline("en", processors='tokenize,pos,lemma,depparse')

2022-05-12 07:20:29 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-05-12 07:20:29 INFO: Use device: cpu
2022-05-12 07:20:29 INFO: Loading: tokenize
2022-05-12 07:20:29 INFO: Loading: pos
2022-05-12 07:20:29 INFO: Loading: lemma
2022-05-12 07:20:29 INFO: Loading: depparse
2022-05-12 07:20:29 INFO: Done loading processors!


In [67]:
import json
import numpy as np
from tqdm import tqdm

def process_sentence(doc):
    token = []
    stanford_pos = []
    stanford_head = []
    stanford_deprel = []
    for sentence in doc.sentences:
        for word in sentence.words:
            token.append(word.text)
            stanford_pos.append(word.xpos)
            stanford_head.append(word.head)
            stanford_deprel.append(word.deprel)
    return token, stanford_pos, stanford_head, stanford_deprel

def find_start_stop(token, token_all):
    # start = token_all.index(token[0])
    # end = token_all.index(end_token)
    possible_start = [i for i, x in enumerate(token_all) if x == token[0]]
    end_token = token[-1]
    
    end = None
    for start in possible_start:
        if token_all[start+len(token)-1] == end_token:
            end = start+len(token)-1
            break

    if end is None:
        print(token_all, token)
        print(possible_start)
        raise Exception("Couldn't find end position")

    return start, end

def check_reverse_relation(obj_start, subj_start):
    return obj_start > subj_start

def create_json_object(index, relation, token, stanford_pos, stanford_head, stanford_deprel, subj_start, subj_end, obj_start, obj_end):
    return {
        "index": index, 
        "relation": relation, 
        "token": token, 
        "stanford_pos": stanford_pos,
        "stanford_head": stanford_head,
        "stanford_deprel": stanford_deprel,
        "subj_start": subj_start,
        "subj_end": subj_end,
        "obj_start": obj_start,
        "obj_end": obj_end
    }

# Generate train data for https://github.com/Cartus/AGGCN

In [68]:
relations = ["AFFECTS","COMPOSITION_MEDIUM","CONTAINS","DEFINED_AS","GENUS","HAS_ATTRIBUTE","HAS_CAUSE","HAS_FORM","HAS_FUNCTION","HAS_LOCATION","HAS_POSITION","HAS_RESULT","HAS_SIZE","MEASURES","OCCURS_IN_MEDIUM","OCCURS_IN_TIME","SPECIES","STUDIES"]
relations_reverse = [r+"_rev" for r in relations]

number_of_rows =df.shape[0]
index = 0
data = []
with tqdm(total=df.shape[0]) as pbar:
    for row_ix, row in df.iterrows():
        sentence_raw = row["SENTENCE"]
        # Edge cases
        sentence_raw = sentence_raw.replace("sediment. of rock", "sediment of rock")
        sentence_raw = sentence_raw.replace("0 ° C.,", "0 ° C,")
        sentence_raw = sentence_raw.replace("(Ford, 2006; Klimchouk, 2007). ", "")

        curr_sentence = nlp(sentence_raw)
        token, stanford_pos, stanford_head, stanford_deprel = process_sentence(curr_sentence)
        obj = row["DEFINIENDUM"]
        
        # Edge cases
        if obj == "lava-cave":
            obj_token = ["lava", "-", "cave"]
        elif obj == "tecto-karstic hollow":
            obj_token = ["tecto-karstic", "hollow"]
        elif obj == "krš":
            obj_token = ["krš"]
        elif obj == "cover-collapse sinkhole":
            obj_token = ["cover", "-collapse", "sinkhole"]
        else:
            obj_token = [w.text for s in nlp(obj).sentences for w in s.words]
        obj_start, obj_end = find_start_stop(obj_token, token)

        for i, relation in enumerate(relations):
            if not pd.isna(row.at[relation]):
                subjects = row[relation].split("|")
                for subject in subjects:
                    # Edge cases
                    subject = subject.replace("e.g .", "e.g.")
                    subject = subject.replace("i.e .", "i.e.")
                    subject = subject.replace("c .", "c.")
                    subject = subject.replace(" ,",",")

                    if subject.startswith(","):
                        subject = subject[1:]
                    elif subject.endswith(" cavities"):
                        subject = subject.replace(" cavities", "")
                    elif subject == "of pure calcium carbonate thin film":
                        subject = "of pure calcium carbonate"

                    if "composite result of carbonate deposition and dissolution" in subject:
                        subj_token = ["composite", "result", "of", "carbonate", "deposition", "and", "dissolution", "cycles", ",", "which", "have", "been", "controlled", "by", "glacial", "sea", "-level", "fluctuations"]
                    elif "extend below sea-level for" in subject:
                        subj_token = ["extend", "below", "sea", "-", "level", "for", "a", "majority", "of", "their", "depth"]
                    elif "in a karst area" in subject:
                        subj_token = ["in", "a", "karst", "area"]
                    else:
                        subj_token = [w.text for s in nlp(subject).sentences for w in s.words]
                    
                    if subj_token[-1] == "etc" and obj=="pipe":
                        subj_token[-1] = "etc."

                    subj_start, subj_end = find_start_stop(subj_token, token)

                    if check_reverse_relation(obj_start, subj_start):
                        relation = relations_reverse[i]

                    curr_json = create_json_object(index, relation, token, stanford_pos, stanford_head, stanford_deprel, subj_start, subj_end, obj_start, obj_end)
                    data.append(curr_json)
                    index += 1
        pbar.update()


print(f"Number of examples: {index}")

json_data = json.dumps(data, indent=4)
os.chdir("/content/drive/MyDrive/1.letnik MAG/NLP/")

with open("karst_stanza_train.json", "w+") as outfile:
    outfile.write(json_data)

100%|██████████| 843/843 [07:18<00:00,  1.92it/s]


Number of examples: 2642


In [69]:
counter = {}
for i in data:
    if i["relation"] not in counter:
        counter[i["relation"]] = 0
    counter[i["relation"]] += 1
couter_sorted = {k: v for k, v in sorted(counter.items(), key=lambda item: item[1], reverse=True)}
print(couter_sorted)

{'GENUS': 629, 'HAS_FORM': 382, 'HAS_LOCATION': 345, 'HAS_CAUSE': 246, 'COMPOSITION_MEDIUM': 175, 'HAS_SIZE': 119, 'HAS_FUNCTION': 110, 'SPECIES': 91, 'CONTAINS': 80, 'GENUS_rev': 78, 'HAS_ATTRIBUTE': 59, 'HAS_FORM_rev': 56, 'DEFINED_AS': 48, 'HAS_LOCATION_rev': 32, 'COMPOSITION_MEDIUM_rev': 31, 'HAS_POSITION': 27, 'HAS_CAUSE_rev': 22, 'HAS_RESULT': 22, 'OCCURS_IN_TIME': 17, 'HAS_FUNCTION_rev': 14, 'HAS_SIZE_rev': 13, 'MEASURES': 11, 'STUDIES': 9, 'CONTAINS_rev': 7, 'SPECIES_rev': 6, 'DEFINED_AS_rev': 4, 'HAS_ATTRIBUTE_rev': 4, 'AFFECTS': 4, 'OCCURS_IN_MEDIUM': 1}


# Generate test data

In [70]:
os.chdir("/content/drive/MyDrive/1.letnik MAG/NLP/NLP-projekt/Data")
df = pd.read_csv('./Test/en/TERM_CAT_GEN_SENT_REL.csv')
df

,DEFINIENDUM,CATEGORY,CATEGORY_TEXT,SENTENCE,AFFECTS,COMPOSITION_MEDIUM,CONTAINS,DEFINED_AS,DEFINITOR,GENUS,...,frame_CAUSE,frame_COMPOSITION,frame_CONTAINS,frame_FORM,frame_FUNCTION,frame_LOCATION,frame_RESULT,frame_SIZE,frame_STUDIES,frame_TIME
0,Erosion,B.2 Loss|B.2 Loss,hydraulic action|Erosion,"Erosion is a hydraulic action, derived from th...",NaN,NaN,NaN,NaN,is a,hydraulic action,...,derived from,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Basin morphometry,E.2 Method|E.2 Method,Basin morphometry|function,Basin morphometry is a function of stream orde...,NaN,NaN,NaN,function of stream order and drainage patterns .,is a,function,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,iceberg,D.1 Abiotic|A.1 Surface landform,piece of ice|iceberg,An iceberg is a piece of ice that has broken f...,NaN,NaN,NaN,NaN,is a,piece of ice,...,NaN,NaN,NaN,NaN,NaN,in the,NaN,NaN,NaN,NaN
3,Abrasion,B. Process|B.2 Loss,process|Abrasion,Abrasion is a particle-by-particle erosion pro...,NaN,NaN,NaN,NaN,is a,process,...,NaN,NaN,NaN,NaN,NaN,NaN,produces,NaN,NaN,NaN
4,aquifer,D.1 Abiotic|A.3 Hydrological form,body of rock|aquifer,An aquifer is defined as a body of rock or unc...,NaN,rock or unconsolidated sediment,NaN,NaN,is defined as a,body of rock|sediment,...,NaN,NaN,NaN,NaN,to allow,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Thermokarst,C. Geome|C. Geome,Thermokarst|terrain,Thermokarst is irregular terrain characterized...,NaN,NaN,topographic depressions with hummocks between ...,NaN,is,terrain,...,NaN,NaN,characterized by,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,Solifluction,B.1 Movement|B. Process,Solifluction|process,Solifluction is an important periglacial proce...,NaN,NaN,NaN,NaN,is an,process,...,NaN,NaN,NaN,NaN,NaN,NaN,forms,NaN,NaN,NaN
101,Dunes,A.1 Surface landform,Dunes,Dunes are collections of loose sand built piec...,NaN,loose sand,NaN,NaN,are,sand,...,by,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,Blowouts,A. Landform|A.1 Surface landform,depressions|Blowouts,Blowouts are depressions created by the deflat...,NaN,NaN,NaN,NaN,are,depressions,...,created by,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
print(df.columns)

Index(['DEFINIENDUM', 'CATEGORY', 'CATEGORY_TEXT', 'SENTENCE', 'AFFECTS',
       'COMPOSITION_MEDIUM', 'CONTAINS', 'DEFINED_AS', 'DEFINITOR', 'GENUS',
       'HAS_ATTRIBUTE', 'HAS_CAUSE', 'HAS_FORM', 'HAS_FUNCTION',
       'HAS_LOCATION', 'HAS_RESULT', 'HAS_SIZE', 'MEASURES', 'OCCURS_IN_TIME',
       'STUDIES', 'frame_ATTRIBUTE', 'frame_CAUSE', 'frame_COMPOSITION',
       'frame_CONTAINS', 'frame_FORM', 'frame_FUNCTION', 'frame_LOCATION',
       'frame_RESULT', 'frame_SIZE', 'frame_STUDIES', 'frame_TIME'],
      dtype='object')


In [72]:
relations = ['AFFECTS', 'COMPOSITION_MEDIUM', 'CONTAINS', 'DEFINED_AS', 'GENUS', 'HAS_ATTRIBUTE', 'HAS_CAUSE', 'HAS_FORM', 'HAS_FUNCTION', 'HAS_LOCATION', 'HAS_RESULT', 'HAS_SIZE', 'MEASURES', 'OCCURS_IN_TIME', 'STUDIES']
relations_reverse = [r+"_rev" for r in relations]

number_of_rows = df.shape[0]
index = 0
data = []
with tqdm(total=df.shape[0]) as pbar:
    for row_ix, row in df.iterrows():
        sentence_raw = row["SENTENCE"]

        curr_sentence = nlp(sentence_raw)
        token, stanford_pos, stanford_head, stanford_deprel = process_sentence(curr_sentence)
        obj = row["DEFINIENDUM"]
        
        obj_token = [w.text for s in nlp(obj).sentences for w in s.words]
        obj_start, obj_end = find_start_stop(obj_token, token)

        for i, relation in enumerate(relations):
            if not pd.isna(row.at[relation]):
                subjects = row[relation].split("|")
                for subject in subjects:
                    subj_token = [w.text for s in nlp(subject).sentences for w in s.words]

                    subj_start, subj_end = find_start_stop(subj_token, token)

                    if check_reverse_relation(obj_start, subj_start):
                        relation = relations_reverse[i]

                    curr_json = create_json_object(index, relation, token, stanford_pos, stanford_head, stanford_deprel, subj_start, subj_end, obj_start, obj_end)
                    data.append(curr_json)
                    index += 1
        pbar.update()


print(f"Number of examples: {index}")

json_data = json.dumps(data, indent=4)
os.chdir("/content/drive/MyDrive/1.letnik MAG/NLP/")

with open("karst_stanza_test.json", "w+") as outfile:
    outfile.write(json_data)

100%|██████████| 104/104 [00:45<00:00,  2.27it/s]


Number of examples: 302


In [73]:
counter = {}
for i in data:
    if i["relation"] not in counter:
        counter[i["relation"]] = 0
    counter[i["relation"]] += 1
counter_sorted = {k: v for k, v in sorted(counter.items(), key=lambda item: item[1], reverse=True)}
print(counter_sorted)

{'GENUS': 113, 'HAS_CAUSE': 35, 'HAS_LOCATION': 30, 'COMPOSITION_MEDIUM': 26, 'HAS_FORM': 22, 'HAS_RESULT': 15, 'HAS_FUNCTION': 14, 'HAS_SIZE': 14, 'HAS_ATTRIBUTE': 12, 'CONTAINS': 7, 'MEASURES': 5, 'DEFINED_AS': 4, 'OCCURS_IN_TIME': 2, 'AFFECTS': 2, 'STUDIES': 1}


# For Slovene

In [7]:
os.chdir("/content/drive/MyDrive/1.letnik MAG/NLP/NLP-projekt/Data")
!pip install classla
import classla

     |████████████████████████████████| 262 kB 14.8 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 1.1 MB 63.2 MB/s 
     |████████████████████████████████| 76 kB 5.8 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x38ef2000 @  0x7ffbcc73f615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 18 kB/s 
     |████████████████████████████████| 262 kB 90.0 MB/s 
  Created wheel for obeliks: filename=obeliks-1.1.5-py3-none-any.whl size=18358 sha256=349730f1c4a3365bd284f2fe6f6bb8a4a962dbd8c38f4b03b86a7dd23c59a292
  Stored in directory: /root/.cache/pip/wheels/16/08/42/75cfe8d6cf9f790e0a864ee62e86138c383d16

In [8]:
classla.download("sl")
nlp_slo = classla.Pipeline('sl', processors='tokenize,pos,lemma,depparse')

2022-05-12 06:06:19 INFO: Downloading these customized packages for language: sl (Slovenian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

2022-05-12 06:07:06 INFO: Finished downloading models and saved to /root/classla_resources.
2022-05-12 06:07:06 INFO: Loading these models for language: sl (Slovenian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |

2022-05-12 06:07:06 INFO: Use device: cpu
2022-05-12 06:07:06 INFO: Loading: tokenize
2022-05-12 06:07:06 INFO: Loading: pos
2022-05-12 06:07:15 INFO: Loading: lemma
2022-05-12 06:07:24 INFO: Loading: depparse
2022-05-12 06:07:26 INFO: Done loading processors!


In [11]:
df = pd.read_csv("./Annotated/SL/TERM_CAT_GEN_SENT_REL.csv")
df

,DEFINIENDUM,CATEGORY,CATEGORY_TEXT,SENTENCE,*,AFFECTS,COMPOSITION_MEDIUM,CONTAINS,DEFINED_AS,DEFINITOR,...,frame_FORM,frame_FUNCTION,frame_LOCATION,frame_MEASURES,frame_MEDIUM,frame_POSITION,frame_RESULT,frame_SIZE,frame_STUDIES,frame_TIME
0,prenikajoča voda,D.1 Abiotic|A.3 Hydrological form|A.3 Hydrolog...,skalne gmote|pronicajoča voda|prenikajoča voda...,"prenikajoča voda, pronicajoča voda: Padavinska...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,v,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,vodni curek,D.1 Abiotic|A.3 Hydrological form|A.3 Hydrolog...,skalne gmote|pronicajoča voda|prenikajoča voda...,"prenikajoča voda, pronicajoča voda: Padavinska...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,v,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,viseča voda,C. Geome|D.1 Abiotic|A.3 Hydrological form|B.1...,aeracijski ( vadozni ) coni|vododržni vložek|v...,viseča voda: Je voda v aeracijski (vadozni) co...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,po
3,piezometrični nivo,D.3 Property|D.1 Abiotic|C. Geome|A.2 Undergro...,piezometrični nivo|vodna gladina|kraškem podze...,"piezometrični nivo, tudi piezometer: Je vsakok...",NaN,NaN,NaN,NaN,Je vsakokratna predpostavljena ploskev,NaN,...,NaN,NaN,v|v,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,piezometer,D.3 Property|D.1 Abiotic|C. Geome|A.2 Undergro...,piezometrični nivo|vodna gladina|kraškem podze...,"piezometrični nivo, tudi piezometer: Je vsakok...",NaN,NaN,NaN,NaN,Je vsakokratna predpostavljena ploskev,NaN,...,NaN,NaN,v|v,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,korozijsko brezno,A.2 Underground landform|B.2 Loss|D.1 Abiotic|...,korozijsko brezno|korozijskim delovanjem|stene...,korozijsko brezno: Nastalo je pretežno s koroz...,NaN,NaN,NaN,NaN,NaN,NaN,...,in ima,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
916,erozijsko brezno,B.4 Transformation|A.2 Underground landform|D....,mehaničnim delovanjem|erozijsko brezno|oblike|...,erozijsko brezno: Nastalo je pretežno z mehani...,NaN,NaN,NaN,in ima jasne erozijske oblike,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
917,poligenetsko brezno,A.2 Underground landform|D.1 Abiotic,poligenetsko brezno|procesi,poligenetsko brezno: Oblikovali so ga različni...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
918,fosilno brezno,A.2 Underground landform|D.1 Abiotic|A.2 Under...,fosilno brezno|speleogenetskih procesov|Brezno...,"fosilno brezno: Brezno, nastalo v preteklih ge...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
relations = ['AFFECTS', 'COMPOSITION_MEDIUM', 'CONTAINS', 'DEFINED_AS', 'GENUS', 'HAS_ATTRIBUTE', 'HAS_CAUSE', 'HAS_FORM', 'HAS_FUNCTION', 'HAS_LOCATION', 'HAS_RESULT', 'HAS_SIZE', 'MEASURES', 'OCCURS_IN_TIME', 'STUDIES']
relations_reverse = [r+"_rev" for r in relations]

number_of_rows = df.shape[0]
index = 0
data = []
with tqdm(total=df.shape[0]) as pbar:
    for row_ix, row in df.iterrows():
        sentence_raw = row["SENTENCE"]

        # EDGE CASES
        sentence_raw = sentence_raw.replace("ob kateri se je razvila. na poklino", "ob kateri se je razvila na poklino")
        sentence_raw = sentence_raw.replace("na podstavku. leži", "na podstavku leži")
        sentence_raw = sentence_raw.replace("(suho dolino, kraško polje). Prim. suha dolina, kraško polje. na Dinarskem krasu", "(suho dolino, kraško polje) na Dinarskem krasu")

        curr_sentence = nlp_slo(sentence_raw)
        token, stanford_pos, stanford_head, stanford_deprel = process_sentence(curr_sentence)
        obj = row["DEFINIENDUM"]
        
        obj_token = [w.text for s in nlp_slo(obj).sentences for w in s.words]
        obj_start, obj_end = find_start_stop(obj_token, token)

        for i, relation in enumerate(relations):
            if not pd.isna(row.at[relation]):

                subjects = row[relation].split("|")
                for subject in subjects:
                    # EDGE CASES
                    subject = subject.replace("ipd .", "ipd.")
                    subject = subject.replace("itd .", "itd.")
                    subject = subject.replace("oz .", "oz.")
                    subject = subject.replace("npr .", "npr.")
                    subject = subject.replace("gl .", "gl.")
                    subject = subject.replace("19 .", "19.")
                    subject = subject.replace("sod .", "sod.")
                    subject = subject.replace("stol .", "stol.")
                    subject = subject.replace("prim .", "prim.")
                    subject = subject.replace("Lj .", "Lj.")
                    subject = subject.replace("v obliki črke V", "v obliki črke V.")
                    subject = subject.replace("od 10 m", "od 10 m.")
                    subject = subject.replace("pri hitrostih , večjih od 10 · 3 m / s", "pri hitrostih , večjih od 10 · 3 m / s.")
                    if subject == "in teče po lastnem nasipu vzdolž gladine podtalnice":
                        # Wrong definition for HAS_LOCATION in sentence 28 and 29
                        continue
        
                    if subject == "Prevaja tudi vode invazijskih jam":
                        subj_token = ['Prevaja', 'tudi', 'vode', 'invazijskih', 'jam.']
                    elif subject == "pretežno zapolnjeno s klastičnimi sedimenti ( Radinja , 1967 , b ) , to je gruščem , prodom , ilovico ipd":
                        subj_token = ['pretežno', 'zapolnjeno', 's', 'klastičnimi', 'sedimenti', '(', 'Radinja', ',', '1967', ',', 'b', ')', ',', 'to', 'je', 'gruščem', ',', 'prodom', ',', 'ilovico', 'ipd.']
                    elif subject == "ki se bavi s preučevanjem jam":
                        subj_token = ['ki', 'se', 'bavi', 's', 'preučevanjem', 'jam.']
                    elif subject == "pretežno zapolnjeno s klastičnimi sedimenti , to je gruščem , prodom , ilovico in pod":
                        subj_token = ['pretežno', 'zapolnjeno', 's', 'klastičnimi', 'sedimenti', ',', 'to', 'je', 'gruščem', ',', 'prodom', ',', 'ilovico', 'in', 'pod.']
                    else:
                        subj_token = [w.text for s in nlp_slo(subject).sentences for w in s.words]

                    subj_start, subj_end = find_start_stop(subj_token, token)

                    if check_reverse_relation(obj_start, subj_start):
                        relation = relations_reverse[i]

                    curr_json = create_json_object(index, relation, token, stanford_pos, stanford_head, stanford_deprel, subj_start, subj_end, obj_start, obj_end)
                    data.append(curr_json)
                    index += 1
        pbar.update()


print(f"Number of examples: {index}")

json_data = json.dumps(data, indent=4)
os.chdir("/content/drive/MyDrive/1.letnik MAG/NLP/")

with open("karst_slo_stanza_train.json", "w+") as outfile:
    outfile.write(json_data)

100%|██████████| 920/920 [10:03<00:00,  1.53it/s]


Number of examples: 2707


In [47]:
counter = {}
for i in data:
    if i["relation"] not in counter:
        counter[i["relation"]] = 0
    counter[i["relation"]] += 1
counter_sorted = {k: v for k, v in sorted(counter.items(), key=lambda item: item[1], reverse=True)}
print(counter_sorted)

{'GENUS': 648, 'HAS_LOCATION': 355, 'HAS_FORM': 343, 'HAS_CAUSE': 281, 'HAS_FUNCTION': 157, 'CONTAINS': 123, 'COMPOSITION_MEDIUM': 115, 'GENUS_rev': 112, 'HAS_SIZE': 111, 'DEFINED_AS': 78, 'HAS_ATTRIBUTE': 75, 'HAS_RESULT': 61, 'OCCURS_IN_TIME': 48, 'HAS_LOCATION_rev': 39, 'HAS_FORM_rev': 32, 'STUDIES': 28, 'HAS_ATTRIBUTE_rev': 15, 'HAS_CAUSE_rev': 15, 'HAS_FUNCTION_rev': 15, 'HAS_SIZE_rev': 13, 'MEASURES': 11, 'COMPOSITION_MEDIUM_rev': 9, 'DEFINED_AS_rev': 8, 'CONTAINS_rev': 6, 'HAS_RESULT_rev': 5, 'OCCURS_IN_TIME_rev': 3, 'AFFECTS': 1}


# Generate test data for slovene

In [49]:
os.chdir("/content/drive/MyDrive/1.letnik MAG/NLP/NLP-projekt/Data")
df = pd.read_csv('./Test/sl/TERM_CAT_GEN_SENT_REL.csv')
df

,DEFINIENDUM,CATEGORY,CATEGORY_TEXT,SENTENCE,AFFECTS,COMPOSITION_MEDIUM,CONTAINS,DEFINED_AS,DEFINITOR,GENUS,...,STUDIES,frame_AFFECTS,frame_CAUSE,frame_COMPOSITION,frame_CONTAINS,frame_FORM,frame_LOCATION,frame_RESULT,frame_SIZE,frame_TIME
0,medledena doba,D.3 Property|A.3 Hydrological form|B.2 Loss|D....,medledena doba|ledenikov|zmanjšanje|interglaci...,Vsako obdobje otoplitve in s tem zmanjšanje ob...,NaN,NaN,NaN,NaN,imenujemo,otoplitve,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,interglacial,D.3 Property|A.3 Hydrological form|B.2 Loss|D....,medledena doba|ledenikov|zmanjšanje|interglaci...,Vsako obdobje otoplitve in s tem zmanjšanje ob...,NaN,NaN,NaN,NaN,imenujemo,otoplitve,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Holocen,A.3 Hydrological form|B.3 Addition|D.3 Property,morske gladine|zviševanjem|Holocen,"Holocen predstavlja medledeno dobo, ki se je z...",NaN,NaN,NaN,NaN,predstavlja,medledeno dobo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,se je začela pred približno
3,pobočni procesi,B. Process|B. Process|A. Landform|B. Process|E...,premeščanja gradiva|pobočni procesi|površja|de...,"Vse pojave premeščanja gradiva, ki nastanejo z...",NaN,NaN,NaN,NaN,imenujemo,pojave,...,NaN,NaN,nastanejo zaradi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pobočni procesi,B. Process|B. Process|A. Landform,pobočni procesi|procese|pobočjih,"Pri izrazu pobočni procesi ime pove, da gre za...",NaN,NaN,NaN,NaN,ime pove,procese,...,NaN,NaN,NaN,NaN,NaN,NaN,na,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Bruhalniki,A.3 Hydrological form|D.1 Abiotic|A.3 Hydrolog...,kraški izviri|vodo|Bruhalniki,"Bruhalniki so kraški izviri, ki obdobno bruhaj...",NaN,NaN,NaN,NaN,so,kraški izviri,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,Korozija,B. Process,Korozija,Korozija je posledica večje količine vode v ni...,NaN,NaN,NaN,NaN,je,NaN,...,NaN,NaN,je posledica,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,Erozijski jarki,A.1 Surface landform|A.1 Surface landform,reliefna oblika|Erozijski jarki,"Erozijski jarki so stalna reliefna oblika, pog...",NaN,NaN,NaN,NaN,so,reliefna oblika,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,Mikrožlebiči,A.1 Surface landform,Mikrožlebiči,"Mikrožlebiči so vzporedne, ravno potekajoče po...",NaN,NaN,NaN,NaN,so,vdolbine,...,NaN,NaN,NaN,NaN,NaN,NaN,se pojavljajo na,NaN,NaN,NaN


In [50]:
print(df.columns)

Index(['DEFINIENDUM', 'CATEGORY', 'CATEGORY_TEXT', 'SENTENCE', 'AFFECTS',
       'COMPOSITION_MEDIUM', 'CONTAINS', 'DEFINED_AS', 'DEFINITOR', 'GENUS',
       'HAS_CAUSE', 'HAS_FORM', 'HAS_FUNCTION', 'HAS_LOCATION', 'HAS_POSITION',
       'HAS_RESULT', 'HAS_SIZE', 'OCCURS_IN_TIME', 'SPECIES', 'STUDIES',
       'frame_AFFECTS', 'frame_CAUSE', 'frame_COMPOSITION', 'frame_CONTAINS',
       'frame_FORM', 'frame_LOCATION', 'frame_RESULT', 'frame_SIZE',
       'frame_TIME'],
      dtype='object')


In [60]:
relations = ['AFFECTS','COMPOSITION_MEDIUM', 'CONTAINS', 'DEFINED_AS', 'GENUS','HAS_CAUSE', 'HAS_FORM', 'HAS_FUNCTION', 'HAS_LOCATION', 'HAS_POSITION','HAS_RESULT', 'HAS_SIZE', 'OCCURS_IN_TIME']
relations_reverse = [r+"_rev" for r in relations]

number_of_rows = df.shape[0]
index = 0
data = []
with tqdm(total=df.shape[0]) as pbar:
    for row_ix, row in df.iterrows():
        sentence_raw = row["SENTENCE"]

        curr_sentence = nlp(sentence_raw)
        token, stanford_pos, stanford_head, stanford_deprel = process_sentence(curr_sentence)
        obj = row["DEFINIENDUM"]
        
        obj_token = [w.text for s in nlp(obj).sentences for w in s.words]
        obj_start, obj_end = find_start_stop(obj_token, token)

        for i, relation in enumerate(relations):
            if not pd.isna(row.at[relation]):
                subjects = row[relation].split("|")
                for subject in subjects:

                    # EDGE CASES
                    subject = subject.replace("npr .", "npr.")

                    subj_token = [w.text for s in nlp(subject).sentences for w in s.words]

                    subj_start, subj_end = find_start_stop(subj_token, token)

                    if check_reverse_relation(obj_start, subj_start):
                        relation = relations_reverse[i]

                    curr_json = create_json_object(index, relation, token, stanford_pos, stanford_head, stanford_deprel, subj_start, subj_end, obj_start, obj_end)
                    data.append(curr_json)
                    index += 1
        pbar.update()


print(f"Number of examples: {index}")

json_data = json.dumps(data, indent=4)
os.chdir("/content/drive/MyDrive/1.letnik MAG/NLP/")

with open("karst_slo_stanza_test.json", "w+") as outfile:
    outfile.write(json_data)

100%|██████████| 113/113 [00:44<00:00,  2.51it/s]

Number of examples: 275


In [61]:
counter = {}
for i in data:
    if i["relation"] not in counter:
        counter[i["relation"]] = 0
    counter[i["relation"]] += 1
counter_sorted = {k: v for k, v in sorted(counter.items(), key=lambda item: item[1], reverse=True)}
print(counter_sorted)

{'GENUS': 76, 'HAS_FORM': 36, 'GENUS_rev': 31, 'HAS_LOCATION': 27, 'HAS_CAUSE': 19, 'HAS_RESULT': 18, 'HAS_SIZE': 13, 'COMPOSITION_MEDIUM': 7, 'HAS_FORM_rev': 7, 'HAS_SIZE_rev': 6, 'HAS_FUNCTION': 6, 'DEFINED_AS': 6, 'HAS_CAUSE_rev': 4, 'HAS_LOCATION_rev': 4, 'HAS_RESULT_rev': 3, 'CONTAINS': 3, 'OCCURS_IN_TIME_rev': 2, 'CONTAINS_rev': 2, 'OCCURS_IN_TIME': 1, 'AFFECTS_rev': 1, 'HAS_POSITION_rev': 1, 'DEFINED_AS_rev': 1, 'COMPOSITION_MEDIUM_rev': 1}
